In [1]:
from numpy.random import randint
from numpy.random import rand
#format of the data 
#[Loan Size(L), interest(rl),rating, loss(lambda)]
#Customers details provided in the question 
customers = [[10,0.021,'AAA',0.0002],
             [25,0.022,'BB',0.0058],
             [4,0.021,'A',0.001],
             [11,0.027,'AA',0.0003],
             [18,0.025,'BBB',0.0024],
             [3,0.026,'AAA',0.0002],
             [17,0.023,'BB',0.0058],
             [15,0.021,'AAA',0.0002],
             [9,0.028,'A',0.001],
             [10,0.022,'A',0.001]             
    
]
D = 60
K = 0.15

In [2]:
# objective function (Basically fitness function)
# rl * L  - lambda +  rt*Transactional cost  - 0.009*D - lambda
def fitness(x):
  sum  = -0.009*D
  for i in range(0,10):
    if x[i] == 1:
      sum = sum + customers[i][1]*customers[i][0] - 2*customers[i][3] + 0.01 * ((1-K)*D - customers[i][0]) 
  return (sum)

In [3]:
#Genetic Validation
def validation(x):
  sum = 0
  for i in range(0,10):
    if x[i] == 1:
      sum = sum + customers[i][0] 
  return (sum)  

In [4]:
#ROULLETE WHEEL SELECTION METHOD
#GENERATING THE CUMULATIVE PROB OF SELECTING THE CHROMOSOME BASED ON FITNESS 
#SELECTING THE INDIVIDUAL USING RANDOM NUMBERS 
def roulette_select(population, num):
    fitnesses = [fitness(i) for i in population]
    total_fitness = float(sum(fitnesses))
    rel_fitness = [f/total_fitness for f in fitnesses]
    probs = [sum(rel_fitness[:i+1]) for i in range(len(rel_fitness))]
    new_population = []
    for n in range(num):
        r = rand()
        for (i, individual) in enumerate(population):
            if r <= probs[i]:
                new_population.append(individual)
                break
    return new_population

In [5]:
#SINGLE POINT CROSS-OVER WHERE R_CROSS IS CROSSOVER PROBABLILITY
#IMPLEMENTED GENETIC VALIDATION AS WELL SO THAT NEWLY GENERATED CHROMOSOMS FOLLOW'S CONSTRAINTS
def crossover(p1, p2, r_cross):
  c1, c2 = p1.copy(), p2.copy() 
  if rand() < r_cross:
    pt = randint(1, len(p1)-2)
    c1 = p1[:pt] + p2[pt:]
    c2 = p2[:pt] + p1[pt:]
  if validation(c1) > (1-K)*D :
    c1 = p1
  if validation(c2) > (1-K)*D :
    c2 = p2
  return [c1, c2]



In [6]:
#IMPLEMENTED MUTATION USING MUTATION PROB.
#IMPLEMENTED GENETIC VALIDATION AS WELL
def mutation(bitstring, r_mut):
  for i in range(len(bitstring)):
    if rand() < r_mut:
      bitstring[i] = 1 - bitstring[i]   
      if validation(bitstring)>((1-K)*D) :
        bitstring[i] = 1 - bitstring[i]



In [7]:
#GENETIC ALGORITHM
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    pop = []
    i = 1
    while i <= n_pop:
        curr = randint(0, 2, n_bits).tolist()
        if validation(curr)<= (1-K)*D:
          pop.append(curr)
          i = i +1
    best,best_eval = 0, objective(pop[0])
    for gen in range(n_iter):
        scores = [objective(c) for c in pop]
        for i in range(n_pop):
          if scores[i] > best_eval:
            best, best_eval = pop[i], scores[i]
        print("Generation %d, New Best fitness(%s) = %.3f" % (gen,  pop[i], scores[i]))
        selected = roulette_select(pop,n_pop)
        children = list()
        for i in range(0, n_pop, 2):
          p1, p2 = selected[i], selected[i+1]
          for c in crossover(p1, p2, r_cross):
            mutation(c, r_mut)
            children.append(c)
        pop = children
    return [best, best_eval]



In [64]:
# define the total iterations
n_iter = 60
# bits
n_bits = 10
# define the population size
n_pop = 8
# crossover rate
r_cross = 0.8
# mutation rate
r_mut = 0.006
best, score = genetic_algorithm(fitness, n_bits, n_iter, n_pop, r_cross, r_mut)
print('GENETIC ALGORITHM IMPLEMENTED')
print('fitness(%s) = %f' % (best, score))

Generation 0, New Best fitness([1, 0, 1, 1, 0, 0, 0, 1, 0, 1]) = 2.631
Generation 1, New Best fitness([1, 0, 1, 0, 0, 0, 0, 1, 0, 0]) = 1.306
Generation 2, New Best fitness([1, 0, 1, 0, 1, 1, 0, 0, 0, 1]) = 2.592
Generation 3, New Best fitness([1, 0, 1, 1, 0, 0, 0, 1, 0, 1]) = 2.631
Generation 4, New Best fitness([1, 0, 1, 1, 0, 0, 0, 1, 1, 0]) = 2.673
Generation 5, New Best fitness([1, 0, 1, 1, 0, 0, 0, 1, 1, 0]) = 2.673
Generation 6, New Best fitness([1, 0, 1, 0, 1, 1, 0, 0, 0, 1]) = 2.592
Generation 7, New Best fitness([0, 0, 1, 0, 1, 0, 0, 1, 0, 0]) = 1.462
Generation 8, New Best fitness([1, 0, 1, 0, 1, 1, 0, 0, 0, 1]) = 2.592
Generation 9, New Best fitness([1, 0, 1, 1, 0, 0, 1, 0, 1, 0]) = 2.717
Generation 10, New Best fitness([0, 0, 1, 1, 0, 0, 1, 0, 1, 0]) = 2.098
Generation 11, New Best fitness([1, 0, 1, 1, 0, 0, 0, 1, 0, 1]) = 2.631
Generation 12, New Best fitness([1, 0, 1, 0, 1, 0, 0, 0, 0, 1]) = 2.035
Generation 13, New Best fitness([1, 0, 1, 1, 0, 0, 1, 0, 1, 0]) = 2.717
Ge

In [65]:
print(best)

[0, 0, 1, 0, 1, 1, 1, 0, 1, 0]


In [67]:
validation(best)

51

In [66]:
print(score)

2.7342000000000004
